In [3]:
import numpy as np
import pandas as pd
import ast

In [4]:
df = pd.read_csv('./evaluations.csv')
df

,topic,payload,device_id,message_id,message_content,timestamp,received_timestamp,payload_size,synthetic_latency,latency,avg_speed,offloading_layer_index,layer_output,device_layers_inference_time
0,devices/,"{""timestamp"":""1727972747.952952546"",""message_i...",device_01,28d7,HelloWorld!,1.727973e+09,1.727973e+09,112,503.434390,-0.045510,-4.888412,NaN,NaN,NaN
1,device_01/model_inference,"{""device_id"": ""edge"", ""message_id"": ""28d7"", ""t...",edge,28d7,AskInference,1.727973e+09,1.727973e+09,632,770.810823,-0.043937,-18.661083,NaN,NaN,NaN
2,devices/,"{""timestamp"":""1727972795.403403396"",""message_i...",device_01,ee6a,HelloWorld!,1.727973e+09,1.727973e+09,112,533.745037,-0.058908,-3.562132,NaN,NaN,NaN
3,device_01/model_inference,"{""device_id"": ""edge"", ""message_id"": ""ee6a"", ""t...",edge,ee6a,AskInference,1.727973e+09,1.727973e+09,632,769.561779,-0.053029,-15.486867,NaN,NaN,NaN
4,device_01/model_inference_result,"{""timestamp"":""1727972795.704704523"",""message_i...",device_01,ee6a,"{'layer_output': ['2.43', '-0.77', '0.54', '1....",1.727973e+09,1.727973e+09,304,1090.297646,-0.063059,-4.421647,"['2.43', '-0.77', '0.54', '1.86', '2.08']",0.0,[0.163666]
5,device_01/end_computation,"{""device_id"": ""edge"", ""message_id"": ""ee6a"", ""t...",edge,ee6a,EndComputation,1.727973e+09,1.727973e+09,115,569.638452,-0.049009,-4.119306,NaN,NaN,NaN


In [5]:
# convert string to list if valid, otherwise assign NaN
def str_to_list(s):
    try:
        return ast.literal_eval(s) if s != "NaN" else np.nan
    except Exception as _:
        return np.nan

# apply the function to the column
df['device_layers_inference_time'] = df['device_layers_inference_time'].apply(str_to_list)
df['edge_layers_inference_time'] = df['edge_layers_inference_time'].apply(str_to_list)
df['layers_sizes'] = df['layers_sizes'].apply(str_to_list)

# define the number of layers (based on the length of the longest list)
n_layers = len(df['layers_sizes'])

# create new columns for each layer
for i in range(n_layers):
    df[f'device_layer_{i + 1}_time'] = df['device_layers_inference_time'].apply(
        lambda x: x[i] if isinstance(x, list) and len(x) > i else None)
    df[f'edge_layer_{i + 1}_time'] = df['edge_layers_inference_time'].apply(
        lambda x: x[i] if isinstance(x, list) and len(x) > i else None)
    df[f'layer_{i + 1}_size'] = df['layers_sizes'].apply(
        lambda x: x[i] if isinstance(x, list) and len(x) > i else None)
# drop unnecessary columns and rows
df.drop(['device_layers_inference_time', 'edge_layers_inference_time', 'layers_sizes'], axis=1, inplace=True)
# drop unnecessary columns
df.drop(['message_content', 'layer_output', 'timestamp', 'recieved_timestamp'], axis=1, inplace=True)

KeyError: 'edge_layers_inference_time'

In [ ]:
df.to_csv('./evaluations_cleaned.csv', index=False)
filtered_df = df[df['topic'] == 'device_01/model_inference_result']
filtered_df.columns

In [ ]:
filtered_df.to_csv('./evaluations_cleaned_v2.csv', index=False)